# Homework 1

#### EE-556 Mathematics of Data - Fall 2024

In this homework, we consider a binary classification task that we will model using logistic regression. Your goal will be to find a classifier using first-order methods and accelerated gradient descent methods. The first part will consist of more theoretical questions, and the second one will ask you to implement these methods.

<div class="alert alert-info">
  ℹ️ <strong>Information on group based work:</strong>
</div>

- You are to deliver only 1 notebook per group.
- Asking assistance beyond your group is ok, but answers should be individual to the group.
- In the event that there was <span style="color: red;">disproportional work done</span> by different group members, let the TAs know.

### Do not forget: Write who are the people in your group as well as their respective SCIPER numbers


Person 1 **Name**: Filippo Passerini || Person 1 **SCIPER**:


Person 2 **Name**: Doan Le Viet Anh|| Person 2 **SCIPER**: 394659 


Person 3 **Name**: Semih Zaman|| Person 3 **SCIPER**:

## 1. Logistic Regression - 15 Points

Logistic regression is a classic approach to _binary classification_. Before we dive in, let us first define the standard logistic function $\sigma$ on which most of what follows is built:

\begin{equation*}
    \sigma : x \mapsto \frac{1}{1 + \exp{(-x)}}.
\end{equation*}

In logistic regression, we model the _conditional probability_ of observing a class label $b$ given a set of features $\mathbf{a}$. More formally, if we observe $n$ independent samples

\begin{equation*}
    \{(\mathbf{a}_i,b_i)\}_{i=1}^n,
\end{equation*}

where $\mathbf{a}_i\in\mathbb{R}^p$ and $b_i\in\{-1, +1\}$ is the class label, we _assume_ that $b_i$ given $\mathbf{a}_i$ is a symmetric Bernouilli random variable with parameter $\sigma(\mathbf{a}_i^T\mathbf{x}^\natural)$, for some unknown $\mathbf{x}^\natural \in \mathbb{R}^p$. In other words, we assume that there exists an $\mathbf{x}^\natural \in \mathbb{R}^p$ such that

\begin{equation*}
    \mathbb{P}(b_i = 1 \mid \mathbf{a}_i) = \sigma(\mathbf{a}_i^T\mathbf{x}^\natural) \quad \text{ and } \quad \mathbb{P}(b_i = -1 \mid \mathbf{a}_i) = 1 - \sigma(\mathbf{a}_i^T\mathbf{x}^\natural)=  \sigma( - \mathbf{a}_i^T\mathbf{x}^\natural).
\end{equation*}

This is our statistical model. It can be written in a more compact form as follows,

\begin{equation*}
    \mathbb{P}(b_i = j \mid \mathbf{a}_i) = \sigma(j \cdot \mathbf{a}_i^T\mathbf{x}^\natural), \quad j \in \{+1, -1\}.
\end{equation*}

Our goal now is to determine the unknown $\mathbf{x}^\natural$ by constructing an estimator.

__(a)__ (1 point) We are provided with a set of $n$ independent observations. Show that the negative log-likelihood $f$ can be written as:

\begin{equation*}
	\begin{aligned}
		f(\mathbf{x}) = -\log(\mathbb{P}(b_1, \dots, b_n | a_1, \dots, a_n)) & = \sum_{i=1}^n  \log(1 + \exp(- b_i \mathbf{a}_i^T\mathbf{x})).
	\end{aligned}
\end{equation*}


**Answer:**
The probability of observing a single data point $(\mathbf{a}_i, b_i)$ is given by:
$$
\mathbb{P}(b_i | \mathbf{a}_i) = \sigma(b_i \mathbf{a}_i^T \mathbf{x}) = \frac{1}{1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x})}, \quad b_i \in \{+1, -1\}.
$$

Since the observations are independent, the likelihood of observing the entire dataset is the product of the individual probabilities:
$$
\mathbb{P}(b_1, \dots, b_n | \mathbf{a}_1, \dots, \mathbf{a}_n) = \prod_{i=1}^n \mathbb{P}(b_i | \mathbf{a}_i) = \prod_{i=1}^n \frac{1}{1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x})}
$$

The log-likelihood is then:
$$
\log(\mathbb{P}(b_1, \dots, b_n | \mathbf{a}_1, \dots, \mathbf{a}_n)) = \sum_{i=1}^n \log \left( \frac{1}{1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x})} \right)
$$

Simplifying the logarithm of the reciprocal, we get:
$$
\log(\mathbb{P}(b_1, \dots, b_n | \mathbf{a}_1, \dots, \mathbf{a}_n)) = -\sum_{i=1}^n \log(1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x}))
$$

Therefore, the negative log-likelihood $f(\mathbf{x})$ is:
$$
f(\mathbf{x}) = -\log(\mathbb{P}(b_1, \dots, b_n | \mathbf{a}_1, \dots, \mathbf{a}_n)) = \sum_{i=1}^n \log(1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x}))
$$

__(b)__ (2 point) Show that the function $u \mapsto \log(1 + \exp(-u))$ is convex. Deduce that $f$ is convex.

**Answer:**

To show that the function $u \mapsto \log(1 + \exp(-u))$ is convex, we need to show that its second derivative is non-negative for all $u \in \mathbb{R}$.

First, let's compute the first derivative of $g(u) = \log(1 + \exp(-u))$:

$$
g'(u) = \frac{d}{du} \log(1 + \exp(-u)) = \frac{-\exp(-u)}{1 + \exp(-u)} = -\sigma(-u)
$$

where $\sigma(u) = \frac{1}{1 + \exp(-u)}$ is the logistic function.

Next, we compute the second derivative:

$$
g''(u) = \frac{d}{du} \left( -\sigma(-u) \right) = \sigma(-u)(1 - \sigma(-u))
$$

Since $\sigma(-u)$ is always between 0 and 1, the product $\sigma(-u)(1 - \sigma(-u))$ is always non-negative. Therefore, $g''(u) \geq 0$ for all $u \in \mathbb{R}$, which implies that $g(u) = \log(1 + \exp(-u))$ is a convex function.

Now, let's deduce that $f(\mathbf{x}) = \sum_{i=1}^n \log(1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x}))$ is convex. Since $g(u) = \log(1 + \exp(-u))$ is convex and the composition of a convex function with an affine function is convex, each term $\log(1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x}))$ is convex in $\mathbf{x}$. 

The sum of convex functions is also convex, so $f(\mathbf{x})$ is convex.

You have just established that the negative log-likelihood is a convex function. So in principle, any local minimum of the maximum likelihood estimator, which is defined as

\begin{equation*}
    \mathbf{x}^\star_{ML} = \arg\min_{\mathbf{x} \in \mathbb{R}^p} f(\mathbf{x}),
\end{equation*}

is a global minimum so it can serve as an estimator of $\mathbf{x}^\natural$. But, does the minimum always exist? We will ponder this question in the following three points.

__(c)__ (1 point) Explain the difference between infima and minima.  Give an example of a convex function, defined over $\mathbb{R}$, that does not attain its infimum. 

#### Infimum vs. Minimum
##### Infimum (inf): 
The greatest lower bound of a set or function values; it may not be attained within the set.
##### Minimum: 
The smallest element in the set; it's the infimum that is actually achieved in the set.
#### Example of a Convex Function Not Attaining Its Infimum
Consider the function $f(x) = e^{x}$ for all $x \in \mathbb{R} $.
##### Convexity:
 $ f $ is convex since its second derivative $ f''(x) = e^{x} > 0 $ everywhere.
##### Infimum:
 $ \inf_{x \in \mathbb{R}} f(x) = 0 $.
##### No Minimum: 
$ f(x) > 0 $ for all $ x $, so $ f $ never attains the value $ 0 $.

__(d)__ (1 point) Let us assume that there exists $\mathbf{x}_0 \in \mathbb{R}^p$ such that 

\begin{equation*}
    \forall i\in\{1, \dots, n\}, \quad \quad b_i \mathbf{a}_i^T\mathbf{x}_0 > 0.
\end{equation*}

This is called _complete separation_ in the literature. Can you think of a geometric reason why this name is appropriate? Think of a 2D example where this can happen (i.e $p=2$) and describe why _complete separation_ is an appropriate name.



The condition is called **complete separation** because it implies that there exists a hyperplane defined by the vector $x_0$ that perfectly divides the data points into two distinct classes without any overlap.

The inequality $b_i \mathbf{a}_i^T \mathbf{x}_0 > 0$ indicates that for each data point $\mathbf{a}_i$, the sign of the projection $\mathbf{a}_i^T \mathbf{x}_0$ aligns with its class label $b_i$.

All points with $b_i = +1$ lie on one side of the hyperplane defined by $\mathbf{a}^T \mathbf{x}_0 > 0$, and all points with $b_i = -1$ lie on the other side. There are no misclassified points or points lying exactly on the hyperplane.


From this, you should see that it is likely that our data satisfies the complete separation assumption. Unfortunately, as you will show in the following question, this can become an obstacle.

__(e)__ (1 point) In a _complete separation_ setting, i.e, there exists $\mathbf{x}_0$ such that 

\begin{equation*}
    \forall i\in\{1, \dots, n\}, \quad \quad b_i \mathbf{a}_i^T\mathbf{x}_0 > 0,
\end{equation*}

prove that the function $f$ does not attain its minimum. 

__Hint__: If the function did have a minimum, would it be above, below or equal to zero? Then think of how $f(2 \mathbf{x}_0)$ compares with $f(\mathbf{x}_0)$, how about $f(\alpha \mathbf{x}_0)$ for $\alpha \rightarrow + \infty$ ? 

**Answer:**

To prove that the function $f$ does not attain its minimum under the condition of complete separation, we need to analyze the behavior of $f(\alpha \mathbf{x}_0)$ as $\alpha \rightarrow +\infty$.

Given the condition of complete separation, we have:
$$
\forall i \in \{1, \dots, n\}, \quad b_i \mathbf{a}_i^T \mathbf{x}_0 > 0.
$$

Let's consider the function $f(\alpha \mathbf{x}_0)$:
$$
f(\alpha \mathbf{x}_0) = \sum_{i=1}^n \log(1 + \exp(-b_i \mathbf{a}_i^T (\alpha \mathbf{x}_0))).
$$

Since $b_i \mathbf{a}_i^T \mathbf{x}_0 > 0$, we have $b_i \mathbf{a}_i^T (\alpha \mathbf{x}_0) = \alpha (b_i \mathbf{a}_i^T \mathbf{x}_0) > 0$ for any $\alpha > 0$. As $\alpha \rightarrow +\infty$, $b_i \mathbf{a}_i^T (\alpha \mathbf{x}_0) \rightarrow +\infty$.

Now, consider the term inside the summation:
$$
\log(1 + \exp(-b_i \mathbf{a}_i^T (\alpha \mathbf{x}_0))).
$$

As $\alpha \rightarrow +\infty$, $-b_i \mathbf{a}_i^T (\alpha \mathbf{x}_0) \rightarrow -\infty$, and thus $\exp(-b_i \mathbf{a}_i^T (\alpha \mathbf{x}_0)) \rightarrow 0$. Therefore,
$$
\log(1 + \exp(-b_i \mathbf{a}_i^T (\alpha \mathbf{x}_0))) \rightarrow \log(1 + 0) = \log(1) = 0.
$$

Hence, as $\alpha \rightarrow +\infty$,
$$
f(\alpha \mathbf{x}_0) \rightarrow \sum_{i=1}^n 0 = 0.
$$

Since $f(\alpha \mathbf{x}_0) \rightarrow 0$ as $\alpha \rightarrow +\infty$, the infimum of $f$ is 0. However, $f(\mathbf{x})$ never actually attains the value 0 for any finite $\mathbf{x}$ because for any finite $\mathbf{x}$, there will always be some positive contribution from the terms $\log(1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x}))$.

Therefore, $f$ does not attain its minimum in the case of complete separation.

As you have just shown convex functions do not always attain their infimum. So it is possible for the maximum-likelihood estimator $\mathbf{x}^\star_{ML}$ to not exist. We will resolve this issue by adding a regularizer. 

In what follows, we consider the function

\begin{equation*}
	f_\mu(\mathbf{x}) = f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2
\end{equation*}
with $\mu> 0$.

__(f)__ (1 point) Show that the gradient of $f_\mu$ can be expressed as 
\begin{equation}
	\nabla f_\mu(\mathbf{x}) = \sum_{i=1}^n -b_i \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i + \mu \mathbf{x}.
\tag{1}
\end{equation}
__Hint__: Lecture 3 shows you how to proceed with this question.

**Answer:**

To show that the gradient of $f_\mu$ can be expressed as 

$$
    \nabla f_\mu(\mathbf{x}) = \sum_{i=1}^n -b_i \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i + \mu \mathbf{x}.
\tag{1}
$$


First, we compute the gradient of $f(\mathbf{x})$. The derivative of $\log(1 + \exp(-u))$ with respect to $u$ is:

$$
    \frac{d}{du} \log(1 + \exp(-u)) = \frac{-\exp(-u)}{1 + \exp(-u)} = -\sigma(-u),
$$

where $\sigma(u) = \frac{1}{1 + \exp(-u)}$ is the logistic function.

Using the chain rule, the gradient of $f(\mathbf{x})$ with respect to $\mathbf{x}$ is:

$$
    \nabla f(\mathbf{x}) = \sum_{i=1}^n \frac{d}{d\mathbf{x}} \log(1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x})) = \sum_{i=1}^n -b_i \sigma(-b_i \mathbf{a}_i^T \mathbf{x}) \mathbf{a}_i.
$$

Next, we compute the gradient of the regularization term $\frac{\mu}{2}\|\mathbf{x}\|_2^2$:

$$
    \nabla \left( \frac{\mu}{2}\|\mathbf{x}\|_2^2 \right) = \mu \mathbf{x}.
$$

Combining these results, the gradient of $f_\mu(\mathbf{x})$ is:

$$
    \nabla f_\mu(\mathbf{x}) = \nabla f(\mathbf{x}) + \nabla \left( \frac{\mu}{2}\|\mathbf{x}\|_2^2 \right) = \sum_{i=1}^n -b_i \sigma(-b_i \mathbf{a}_i^T \mathbf{x}) \mathbf{a}_i + \mu \mathbf{x}.
$$


__(g)__ (1 point) Show that the Hessian of $f_\mu$ can be expressed as 
\begin{equation}
	\nabla^2 f_\mu(\mathbf{x}) = \sum_{i=1}^{n} \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}))\mathbf{a}_i\mathbf{a}_i^T + \mu \mathbf{I}.
\tag{2}
\end{equation}

**Answer:**

To show that the Hessian of $f_\mu$ can be expressed as 

$$
    \nabla^2 f_\mu(\mathbf{x}) = \sum_{i=1}^{n} \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}))\mathbf{a}_i\mathbf{a}_i^T + \mu \mathbf{I},
\tag{2}
$$


First, we compute the Hessian of $f(\mathbf{x})$. The gradient of $f(\mathbf{x})$ is:

$$
    \nabla f(\mathbf{x}) = \sum_{i=1}^n -b_i \sigma(-b_i \mathbf{a}_i^T \mathbf{x}) \mathbf{a}_i.
$$

To find the Hessian, we need to differentiate the gradient again with respect to $\mathbf{x}$. The derivative of $\sigma(u)$ with respect to $u$ is:

$$
    \frac{d}{du} \sigma(u) = \sigma(u)(1 - \sigma(u)).
$$

Using the chain rule, the second derivative of $f(\mathbf{x})$ with respect to $\mathbf{x}$ is:

$$
    \nabla^2 f(\mathbf{x}) = \sum_{i=1}^n b_i^2 \sigma(-b_i \mathbf{a}_i^T \mathbf{x})(1 - \sigma(-b_i \mathbf{a}_i^T \mathbf{x})) \mathbf{a}_i \mathbf{a}_i^T.
$$

Since $b_i^2 = 1$ for $b_i \in \{-1, +1\}$, we have:

$$
    \nabla^2 f(\mathbf{x}) = \sum_{i=1}^n \sigma(-b_i \mathbf{a}_i^T \mathbf{x})(1 - \sigma(-b_i \mathbf{a}_i^T \mathbf{x})) \mathbf{a}_i \mathbf{a}_i^T.
$$

Next, we compute the Hessian of the regularization term $\frac{\mu}{2}\|\mathbf{x}\|_2^2$:

$$
    \nabla^2 \left( \frac{\mu}{2}\|\mathbf{x}\|_2^2 \right) = \mu \mathbf{I}.
$$

Combining these results, the Hessian of $f_\mu(\mathbf{x})$ is:

$$
    \nabla^2 f_\mu(\mathbf{x}) = \nabla^2 f(\mathbf{x}) + \nabla^2 \left( \frac{\mu}{2}\|\mathbf{x}\|_2^2 \right) = \sum_{i=1}^n \sigma(-b_i \mathbf{a}_i^T \mathbf{x})(1 - \sigma(-b_i \mathbf{a}_i^T \mathbf{x})) \mathbf{a}_i \mathbf{a}_i^T + \mu \mathbf{I}.
$$


It is convenient to observe that we can write the Hessian in a more compact form by defining the matrix
\begin{equation}
	\mathbf{A} = \begin{bmatrix}
        \leftarrow &  \mathbf{a}_1^T & \rightarrow \\
        \leftarrow &  \mathbf{a}_2^T & \rightarrow \\
         &  \ldots &  \\
        \leftarrow &  \mathbf{a}_n^T & \rightarrow \\
  \end{bmatrix}.
\end{equation}
It is easy to see that we have
\begin{equation}
	\nabla^2 f_\mu(\mathbf{x}) =  \mathbf{A}^T \text{Diag}\left(\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})) \right)\mathbf{A}+ \mu \mathbf{I}.
\end{equation}


__(h)__ (1 point) Show that $f_\mu$ is $\mu$-strongly convex. 

**Answer:**

To show that $f_\mu$ is $\mu$-strongly convex, we need to show that for all $\mathbf{x}, \mathbf{y} \in \mathbb{R}^p$ and $\alpha \in [0, 1]$, the following inequality holds:

$$
f_\mu(\alpha \mathbf{x} + (1 - \alpha) \mathbf{y}) \leq \alpha f_\mu(\mathbf{x}) + (1 - \alpha) f_\mu(\mathbf{y}) - \frac{\mu}{2} \alpha (1 - \alpha) \|\mathbf{x} - \mathbf{y}\|_2^2.
$$

Recall that $f_\mu(\mathbf{x}) = f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2$. Since $f(\mathbf{x})$ is convex, we have:

$$
f(\alpha \mathbf{x} + (1 - \alpha) \mathbf{y}) \leq \alpha f(\mathbf{x}) + (1 - \alpha) f(\mathbf{y}).
$$

Now, consider the regularization term $\frac{\mu}{2}\|\mathbf{x}\|_2^2$. This term is $\mu$-strongly convex because the function $\frac{\mu}{2}\|\mathbf{x}\|_2^2$ is quadratic and its Hessian is $\mu \mathbf{I}$, which is positive definite with eigenvalues equal to $\mu$.

Therefore, for the regularization term, we have:

$$
\frac{\mu}{2}\|\alpha \mathbf{x} + (1 - \alpha) \mathbf{y}\|_2^2 \leq \alpha \frac{\mu}{2}\|\mathbf{x}\|_2^2 + (1 - \alpha) \frac{\mu}{2}\|\mathbf{y}\|_2^2 - \frac{\mu}{2} \alpha (1 - \alpha) \|\mathbf{x} - \mathbf{y}\|_2^2.
$$

Combining the convexity of $f(\mathbf{x})$ and the $\mu$-strong convexity of the regularization term, we get:

$$
f_\mu(\alpha \mathbf{x} + (1 - \alpha) \mathbf{y}) = f(\alpha \mathbf{x} + (1 - \alpha) \mathbf{y}) + \frac{\mu}{2}\|\alpha \mathbf{x} + (1 - \alpha) \mathbf{y}\|_2^2
$$

$$
\leq \alpha f(\mathbf{x}) + (1 - \alpha) f(\mathbf{y}) + \alpha \frac{\mu}{2}\|\mathbf{x}\|_2^2 + (1 - \alpha) \frac{\mu}{2}\|\mathbf{y}\|_2^2 - \frac{\mu}{2} \alpha (1 - \alpha) \|\mathbf{x} - \mathbf{y}\|_2^2
$$

$$
= \alpha f_\mu(\mathbf{x}) + (1 - \alpha) f_\mu(\mathbf{y}) - \frac{\mu}{2} \alpha (1 - \alpha) \|\mathbf{x} - \mathbf{y}\|_2^2.
$$

Thus, $f_\mu$ is $\mu$-strongly convex.

__(i)__ (3 points) Is it possible for a strongly convex function to not attain its minimum ? <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) Justify your reasoning.

We will now show that $f_\mu$ is smooth, i.e, $\nabla f_\mu$ is L-Lipschitz with respect to the Euclidean norm, with 
\begin{equation}
	L = \|A\|^2_F + \mu \text{, where }\|\cdot\|_F\text{ denotes the Frobenius norm. }
\end{equation}

(1 point for all three questions)

__(j-1)__ Show that $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) = \left\| \mathbf{a}_i\right\|_2^2$, where $\lambda_{\max}(\cdot)$ denotes the largest eigenvalue.


To show that $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) = \left\| \mathbf{a}_i\right\|_2^2$, where $\lambda_{\max}(\cdot)$ denotes the largest eigenvalue, we proceed as follows:

Consider the matrix $\mathbf{a}_i\mathbf{a}_i^T$. This is an outer product of the vector $\mathbf{a}_i$ with itself. The matrix $\mathbf{a}_i\mathbf{a}_i^T$ is symmetric and positive semi-definite.

1. **Eigenvalues of $\mathbf{a}_i\mathbf{a}_i^T$:**

    Let $\mathbf{v}$ be an eigenvector of $\mathbf{a}_i\mathbf{a}_i^T$ with eigenvalue $\lambda$. Then,
    $$
    \mathbf{a}_i\mathbf{a}_i^T \mathbf{v} = \lambda \mathbf{v}.
    $$

2. **Form of $\mathbf{a}_i\mathbf{a}_i^T$:**

    Notice that $\mathbf{a}_i\mathbf{a}_i^T$ is a rank-1 matrix, since it is formed by the outer product of $\mathbf{a}_i$ with itself. Therefore, it has at most one non-zero eigenvalue.

3. **Non-zero eigenvalue:**

    To find the non-zero eigenvalue, consider $\mathbf{v} = \mathbf{a}_i$. Then,
    $$
    \mathbf{a}_i\mathbf{a}_i^T \mathbf{a}_i = \mathbf{a}_i (\mathbf{a}_i^T \mathbf{a}_i) = \|\mathbf{a}_i\|_2^2 \mathbf{a}_i.
    $$
    This shows that $\mathbf{a}_i$ is an eigenvector of $\mathbf{a}_i\mathbf{a}_i^T$ with eigenvalue $\|\mathbf{a}_i\|_2^2$.

4. **Conclusion:**

    Since $\mathbf{a}_i\mathbf{a}_i^T$ is a rank-1 matrix, the only non-zero eigenvalue is $\|\mathbf{a}_i\|_2^2$. Therefore,
    $$
    \lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) = \|\mathbf{a}_i\|_2^2.
    $$


__(j-2)__ Using [2](#mjx-eqn-eq2), show that $\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu$. 

__Hint__: Recall that $\lambda_{\max}(\cdot)$ verifies the triangle inequality.

To show that $\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu$, we start with the expression for the Hessian of $f_\mu(\mathbf{x})$:

$$
\nabla^2 f_\mu(\mathbf{x}) = \sum_{i=1}^{n} \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}))\mathbf{a}_i\mathbf{a}_i^T + \mu \mathbf{I}.
$$

Recall that for any matrices $\mathbf{B}$ and $\mathbf{C}$, the largest eigenvalue of their sum satisfies the triangle inequality:

$$
\lambda_{\max}(\mathbf{B} + \mathbf{C}) \leq \lambda_{\max}(\mathbf{B}) + \lambda_{\max}(\mathbf{C}).
$$

Applying this to our Hessian, we get:

$$
\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \lambda_{\max}\left(\sum_{i=1}^{n} \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}))\mathbf{a}_i\mathbf{a}_i^T\right) + \lambda_{\max}(\mu \mathbf{I}).
$$

The largest eigenvalue of $\mu \mathbf{I}$ is simply $\mu$, since $\mathbf{I}$ is the identity matrix. Therefore,

$$
\lambda_{\max}(\mu \mathbf{I}) = \mu.
$$

Next, we consider the term $\sum_{i=1}^{n} \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}))\mathbf{a}_i\mathbf{a}_i^T$. Each term $\mathbf{a}_i\mathbf{a}_i^T$ is a rank-1 matrix, and its largest eigenvalue is $\|\mathbf{a}_i\|_2^2$ as shown in the previous part. The scalar $\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}))$ is always between 0 and 1, so it does not increase the eigenvalue.

Therefore,

$$
\lambda_{\max}\left(\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}))\mathbf{a}_i\mathbf{a}_i^T\right) \leq \|\mathbf{a}_i\|_2^2.
$$

Summing over all $i$, we get:

$$
\lambda_{\max}\left(\sum_{i=1}^{n} \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}))\mathbf{a}_i\mathbf{a}_i^T\right) \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2.
$$

Combining these results, we have:

$$
\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu.
$$

__(j-3)__ Conclude that $f_\mu$ is $L$-smooth for $L = \|A\|_F^2 + \mu$.


To conclude that $f_\mu$ is $L$-smooth for $L = \|A\|_F^2 + \mu$, we need to show that the gradient $\nabla f_\mu(\mathbf{x})$ is Lipschitz continuous with Lipschitz constant $L$.

A function $f_\mu$ is $L$-smooth if its gradient $\nabla f_\mu(\mathbf{x})$ satisfies the following inequality for all $\mathbf{x}, \mathbf{y} \in \mathbb{R}^p$:

$$
\|\nabla f_\mu(\mathbf{x}) - \nabla f_\mu(\mathbf{y})\|_2 \leq L \|\mathbf{x} - \mathbf{y}\|_2.
$$

From the previous results, we have shown that the largest eigenvalue of the Hessian $\nabla^2 f_\mu(\mathbf{x})$ is bounded by $\sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu$. This implies that the Hessian is bounded as follows:

$$
\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu.
$$

The Frobenius norm of the matrix $\mathbf{A}$, denoted as $\|A\|_F$, is defined as:

$$
\|A\|_F^2 = \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2.
$$

Therefore, we can rewrite the bound on the largest eigenvalue of the Hessian as:

$$
\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \|A\|_F^2 + \mu.
$$

Since the largest eigenvalue of the Hessian provides an upper bound on the Lipschitz constant of the gradient, we conclude that $f_\mu$ is $L$-smooth with $L = \|A\|_F^2 + \mu$.

__(l)__ (2 point) To finalize, we introduce the Kullback-Leibler (KL) divergence. The KL divergence is a measure of how one probability distribution differs from a second, reference probability distribution. Along side the examples we saw in slide 18 of Lecture 1, the KL divergence is also a useful loss function to be used in learning frameworks.

Write the definition of the Kullback-Leibler (KL) divergence between the true label distribution $q(b_i)$ and the model’s predicted distribution $p(b_i∣\mathbf{a}_i)$ and show that minimizing the KL divergence between $q(b_i)$ and $p(b_i∣\mathbf{a}_i)$ is equivalent to minimizing the negative log-likelihood derived in (a).

The Kullback-Leibler (KL) divergence between the true label distribution $q(b_i)$ and the model’s predicted distribution $p(b_i \mid \mathbf{a}_i)$ is defined as:

$$
D_{KL}(q(b_i) \parallel p(b_i \mid \mathbf{a}_i)) = \sum_{b_i} q(b_i) \log \left( \frac{q(b_i)}{p(b_i \mid \mathbf{a}_i)} \right).
$$

In the context of binary classification with logistic regression, the true label distribution $q(b_i)$ can be represented as a Bernoulli distribution with parameter $b_i$, and the model's predicted distribution $p(b_i \mid \mathbf{a}_i)$ is given by the logistic function $\sigma(\mathbf{a}_i^T \mathbf{x})$.

For binary labels $b_i \in \{-1, +1\}$, the KL divergence can be written as:

$$
D_{KL}(q(b_i) \parallel p(b_i \mid \mathbf{a}_i)) = q(b_i = 1) \log \left( \frac{q(b_i = 1)}{p(b_i = 1 \mid \mathbf{a}_i)} \right) + q(b_i = -1) \log \left( \frac{q(b_i = -1)}{p(b_i = -1 \mid \mathbf{a}_i)} \right).
$$

Since $q(b_i = 1) = \sigma(\mathbf{a}_i^T \mathbf{x})$ and $q(b_i = -1) = 1 - \sigma(\mathbf{a}_i^T \mathbf{x})$, we have:

$$
D_{KL}(q(b_i) \parallel p(b_i \mid \mathbf{a}_i)) = \sigma(\mathbf{a}_i^T \mathbf{x}) \log \left( \frac{\sigma(\mathbf{a}_i^T \mathbf{x})}{p(b_i = 1 \mid \mathbf{a}_i)} \right) + (1 - \sigma(\mathbf{a}_i^T \mathbf{x})) \log \left( \frac{1 - \sigma(\mathbf{a}_i^T \mathbf{x})}{p(b_i = -1 \mid \mathbf{a}_i)} \right).
$$

The predicted probabilities $p(b_i = 1 \mid \mathbf{a}_i)$ and $p(b_i = -1 \mid \mathbf{a}_i)$ are given by:

$$
p(b_i = 1 \mid \mathbf{a}_i) = \sigma(\mathbf{a}_i^T \mathbf{x}),
$$

$$
p(b_i = -1 \mid \mathbf{a}_i) = 1 - \sigma(\mathbf{a}_i^T \mathbf{x}).
$$

Substituting these into the KL divergence formula, we get:

$$
D_{KL}(q(b_i) \parallel p(b_i \mid \mathbf{a}_i)) = \sigma(\mathbf{a}_i^T \mathbf{x}) \log \left( \frac{\sigma(\mathbf{a}_i^T \mathbf{x})}{\sigma(\mathbf{a}_i^T \mathbf{x})} \right) + (1 - \sigma(\mathbf{a}_i^T \mathbf{x})) \log \left( \frac{1 - \sigma(\mathbf{a}_i^T \mathbf{x})}{1 - \sigma(\mathbf{a}_i^T \mathbf{x})} \right).
$$

Simplifying, we see that the KL divergence reduces to the negative log-likelihood:

$$
D_{KL}(q(b_i) \parallel p(b_i \mid \mathbf{a}_i)) = -\log(p(b_i \mid \mathbf{a}_i)).
$$

Therefore, minimizing the KL divergence between $q(b_i)$ and $p(b_i \mid \mathbf{a}_i)$ is equivalent to minimizing the negative log-likelihood derived in (a).


From your work in this section, you have shown that the maximum likelihood estimator for logistic regression might not exist, but it can be guaranteed to exist by adding a $\|\cdot\|_2^2$ regularizer. Consequently, the estimator for $\mathbf{x}^\natural$ we will use will be the solution of the smooth strongly convex problem,
\begin{equation}
	\mathbf{x}^\star=\arg\min_{\mathbf{x} \in \mathbb{R}^p} f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2.
\tag{3}
\end{equation}

<a name="cite_note-1"></a>1. [^](#cite_ref-1) TA's will give you candy if you provide a complete proof.